# Import des packages

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

#### Import de la base de donnée

In [2]:
data = pd.read_csv('QueryResults_2.csv')

# Data preparation for text analysis

### Fonctions primaires 

In [3]:
# Lemmatizer, permet de regrouper les mots dont le sens est très liés 
#(différentes conjugaisons d'un même verbe, singulier et pluriel du même nom).
def lemma_fct(list_words) :
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w
"""
#Tokenizer, permet de tokenizer les phrases, ou texte, en liste de mot. 
def tokenizer_fct(question) :
    # print(sentence)
    question = BeautifulSoup(question).get_text()
    question = question.lower()
    #sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(question)
    return word_tokens
# stopwords remover, permet de supprimer les mots à faible valeur ajoutée (les mots très courants)
def stop_word_filter_fct(list_words) :
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2
"""

"\n#Tokenizer, permet de tokenizer les phrases, ou texte, en liste de mot. \ndef tokenizer_fct(question) :\n    # print(sentence)\n    question = BeautifulSoup(question).get_text()\n    question = question.lower()\n    #sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')\n    word_tokens = word_tokenize(question)\n    return word_tokens\n# stopwords remover, permet de supprimer les mots à faible valeur ajoutée (les mots très courants)\ndef stop_word_filter_fct(list_words) :\n    filtered_w = [w for w in list_words if not w in stop_w]\n    filtered_w2 = [w for w in filtered_w if len(w) > 2]\n    return filtered_w2\n"

### Fonction déstinée à la création d'un "Bag of Word"

In [4]:
def question_to_words_BW(question):
    # On récupère le texte
    question_text = BeautifulSoup(question).get_text()
    
    #On ne garde que les caractères intéressants. 
    question = re.sub("[^a-zA-Z+#]"," ", question_text)
    
    # On tokenize nos questions en mots.
    words = question.lower().split()
    
    # On retire les mots clés plus courants avec peu de valeur ajoutée. 
    stops = set(stopwords.words('english'))
    meaningful_words = [w for w in words if not w in stops]
        
    # On lemmatize les mots
    meaningful_words = lemma_fct(meaningful_words)
    
    #On retourne une liste comprenant les mots ainsi récupérés. 
    return(" ".join( meaningful_words))

# BAG Of WORDS

### Préprocessing

In [5]:
questions_cleared = []
for question in data.Body:
        cleared_question = question_to_words_BW(question)
        questions_cleared.append(cleared_question)
    
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# On créé le vectorizer qui va vectorizer les mots de nos questions. 

#vectorizer = CountVectorizer(analyzer = "word", max_features = 5000) 
vectorizer = TfidfVectorizer(analyzer = "word", max_features = 5000)
    
# On fit notre vectorizer avec nos questions. 
questions_features = vectorizer.fit_transform(questions_cleared)
#questions_features = questions_features.toarray()

### Modèle de prédiction

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20

lda = LatentDirichletAllocation(n_components = n_topics, max_iter = 5, learning_method='online',learning_offset=50.,random_state=0)

lda.fit(questions_features)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=20, random_state=0)

### Affichage des topics trouvés par notre modèle.


In [7]:
def display_topics(model, feature_names, no_top_words):
    topic_list = pd.DataFrame()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        topic_list[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

no_top_words = 5
display_topics(lda,vectorizer.get_feature_names(), 5)

Topic 0:
socket firebase sd room cloud
Topic 1:
image div android text button
Topic 2:
git branch commit repository github
Topic 3:
nsdate stdout pipe subprocess scanner
Topic 4:
plot np numpy matplotlib plt
Topic 5:
foo bar baz vue john
Topic 6:
excel sheet dog mongoose animal
Topic 7:
cmake beautifulsoup closure analyzer soup
Topic 8:
utf character encoding mm iframe
Topic 9:
public user new class request
Topic 10:
scala rdd claim ipsum lorem
Topic 11:
largest portaudiomodule libsass addrreturn dealloc
Topic 12:
file project command error run
Topic 13:
timer tweet favicon fullscreen trans
Topic 14:
urllib bob executor urlopen executorservice
Topic 15:
table column id query sql
Topic 16:
string int array function list
Topic 17:
would use like one using
Topic 18:
self view nsstring nil swift
Topic 19:
hash key sha byte rsa


/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
from __future__ import print_function
import pyLDAvis

import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, questions_features, vectorizer)

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mod

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.145932  0.102397       1        1  29.753538
9      0.180004  0.063073       2        1  21.328205
12     0.194460  0.085261       3        1  12.996273
1      0.245146 -0.002382       4        1   9.381429
16     0.264743  0.016569       5        1   9.301268
15     0.269899 -0.157979       6        1   3.113140
18    -0.051502 -0.356634       7        1   1.548685
2     -0.088850 -0.165826       8        1   1.294506
8     -0.095753 -0.042667       9        1   1.086538
4     -0.113474 -0.028857      10        1   1.071322
19    -0.095166  0.005767      11        1   1.004966
5     -0.114645  0.007772      12        1   0.993958
6     -0.114114  0.037235      13        1   0.961238
3     -0.100384  0.058656      14        1   0.904923
0     -0.102036  0.057822      15        1   0.904884
14    -0.101144  0.059662      16        1   0.899516
7     -0.088909  0.063075      17        1   0.875187
10    -0.085246  0.065427      18        1   0.871214
13    -0.082443  0.065855      19        1   0.865129
11    -0.066518  0.065775      20        1   0.844082, topic_info=                 Term        Freq       Total Category  logprob  loglift
1705              foo  664.000000  664.000000  Default  30.0000  30.0000
1849              git  455.000000  455.000000  Default  29.0000  29.0000
2374              key  527.000000  527.000000  Default  28.0000  28.0000
372               bar  436.000000  436.000000  Default  27.0000  27.0000
3919             self  479.000000  479.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
2202           insert    0.400741  129.125686  Topic20  -8.5407  -1.0005
1934              gyp    0.400741   10.547321  Topic20  -8.5407   1.5044
211   applicationuser    0.400741    9.938413  Topic20  -8.5407   1.5638
1926             guid    0.400741   26.395077  Topic20  -8.5407   0.5871
3358          primary    0.400741   70.690564  Topic20  -8.5407  -0.3981

[1058 rows x 6 columns], token_table=      Topic      Freq                                Term
term                                                     
1         2  0.057263                                 aaa
1         6  0.744419                                 aaa
13        2  0.382265  abstractautowirecapablebeanfactory
14        2  0.211482                 abstractbeanfactory
55        1  0.150951                            activity
...     ...       ...                                 ...
4993      3  0.226270                                zlib
4993     11  0.633555                                zlib
4996      4  0.910625                                zoom
4998     10  0.896590                                 zsh
4999     10  0.861399                          zygoteinit

[2120 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 10, 13, 2, 17, 16, 19, 3, 9, 5, 20, 6, 7, 4, 1, 15, 8, 11, 14, 12])

### Création de la Pipeline regroupant le préprocessing et le modèle LDA

In [9]:
# On créé le vectorizer qui va vectorizer les mots de nos questions. 

# Pour chaque question, on applique la fonction qui récupère les mots clés des questions posées. 
def clearing_questions(Body):
    # On créé une liste vide déstinée à receullir les mots de nos questions. 
    questions_cleared =[]
    for question in Body:
        cleared_question = question_to_words_BW(question)
        questions_cleared.append(cleared_question)
        
    return questions_cleared

In [10]:
from sklearn import datasets, preprocessing, model_selection, ensemble, pipeline
from sklearn.experimental import enable_hist_gradient_boosting

/Users/florentmargery/opt/anaconda3/envs/work_env/lib/python3.9/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [12]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline

n_topics = 20
transformer_1 = FunctionTransformer(clearing_questions)
"""
pipeline_BOW = Pipeline([
   ('transformer', transformer_1), ('vectorizer',CountVectorizer(analyzer = "word", max_features = 5000)),
   ('LDA',LatentDirichletAllocation(n_components = n_topics, max_iter = 5,\
                                                                     learning_method='online',learning_offset=50.,random_state=0))
])
"""

pipeline_BOW = Pipeline([
   ('transformer', transformer_1), ('vectorizer',TfidfVectorizer(analyzer = "word", max_features = 5000)),
   ('LDA',LatentDirichletAllocation(n_components = n_topics, max_iter = 5,\
                                                                     learning_method='online',learning_offset=50.,random_state=0))
])

In [13]:
pipeline_BOW.fit(data.Body)

Pipeline(steps=[('transformer',
                 FunctionTransformer(func=<function clearing_questions at 0x7fe6b84f1f70>)),
                ('vectorizer', TfidfVectorizer(max_features=5000)),
                ('LDA',
                 LatentDirichletAllocation(learning_method='online',
                                           learning_offset=50.0, max_iter=5,
                                           n_components=20, random_state=0))])

In [14]:
def create_topic_list(model, feature_names, no_top_words):
    topic_list = pd.DataFrame()
    for topic_idx, topic in enumerate(model.components_):
        topic_list[topic_idx] = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    return topic_list

def tag_generator(questions):
    for question in questions:
        new_document_topic = np.where(question == question.max())
        new_document_topic = int(new_document_topic[0])
        print('La répartition des proba d\'appartenance aux différents TOPICS est \n  ', question)
        print('Les tags suggérés pour ce document sont :  \n',topic_list.T.iloc[new_document_topic,:5])
    return topic_list.T.iloc[new_document_topic,:5].tolist()



In [15]:
no_top_words = 5
topic_list = create_topic_list(pipeline_BOW['LDA'],pipeline_BOW['vectorizer'].get_feature_names_out(),no_top_words)

### Essai de la pipeline avant API

In [16]:
test_text = pipeline_BOW.transform([data.Body[10]])
tag_generator(test_text)

La répartition des proba d'appartenance aux différents TOPICS est 
   [0.08098836 0.0085541  0.0085541  0.0085541  0.0085541  0.0085541
 0.0085541  0.0085541  0.0085541  0.0085541  0.0085541  0.0085541
 0.0085541  0.0085541  0.0085541  0.10318764 0.0085541  0.67040431
 0.0085541  0.0085541 ]
Les tags suggérés pour ce document sont :  
 0    would
1      use
2     like
3      one
4    using
Name: 17, dtype: object


['would', 'use', 'like', 'one', 'using']

In [17]:
data.Body[10]

"<p>In our place we're split between using mysqli and PDO for stuff like prepared statements and transaction support. Some projects use one, some the other. There is little realistic likelihood of us ever moving to another RDBMS.</p>\n\n<p>I prefer PDO for the single reason that it allows named parameters for prepared statements, and as far as I am aware mysqli does not.</p>\n\n<p>Are there any other pros and cons to choosing one over the other as a standard as we consolidate our projects to use just one approach?</p>\n"

### Création de "Topic List'

# API BagOfWord

In [18]:
from fast_dash import FastDash
from fast_dash.Components import Text

# Step 1: Define your model inference
def text_to_text_function(question):
    
    result = pipeline_BOW.transform([question])
    result = tag_generator(result)
    
    return result

# Step 2: Specify the input and output components
app = FastDash(callback_fn=text_to_text_function, 
                inputs=Text, 
                outputs=Text, 
                title='Unsupervised Question TAG Generator')

# Step 3: Run your app!
app.run()

# * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)

 * Serving Flask app 'fast_dash.fast_dash' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2022 17:08:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:46] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:46] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:46] "GET /assets/favicon.ico?m=1649961764.648881 HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:08:49] "POST /_dash-up

127.0.0.1 - - [19/Apr/2022 17:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.

La répartition des proba d'appartenance aux différents TOPICS est 
   [0.01187696 0.01187696 0.01187696 0.01187696 0.01187696 0.01187696
 0.01187696 0.01187696 0.01187696 0.01187696 0.01187696 0.01187696
 0.77433777 0.01187696 0.01187696 0.01187696 0.01187697 0.01187696
 0.01187696 0.01187696]
Les tags suggérés pour ce document sont :  
 0       file
1    project
2    command
3      error
4        run
Name: 12, dtype: object


127.0.0.1 - - [19/Apr/2022 17:09:33] "GET /assets/favicon.ico?m=1649961764.648881 HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2022 17:09:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:34] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2022 17:09:34] "GET /assets/favicon.ico?m=1649961764.648881 HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2022 17:09:34] "POST /_dash-update-component HTTP/1.1" 200 -
